## DATA CURATION
---

The goal of this section of the project is to gather more detailed information about the titles that exist in our dataset 'vgsales.csv'. This dataset contains more than 10,000 titles, but not much else in terms of descriptive features of the titles included. We will use the titles from this dataset as the basis for building a more robust one, which we will refer to from now on as the 'game library'.

In [3]:
# necessary libraries/modules
import os.path

import pandas as pd
import numpy as np
import requests
import json
import csv
from pathlib import Path
from importlib import reload
from os import listdir, getcwd, path
from typing import List, Any, Dict, Optional
from datetime import datetime, date
from itertools import islice
from datetime import timedelta
from ratelimit import limits, sleep_and_retry

In [4]:
# local py files
import data_pull
from data_pull import *
from api_auth_keys import *
reload(data_pull)

<module 'data_pull' from 'C:\\Users\\Matt\\DataspellProjects\\VGRecomendation\\data_pull.py'>

In [5]:
# data paths
datasets = Path(getcwd()).joinpath('datasets')
library = datasets.joinpath('library.csv')
new_data = datasets.joinpath('new_data.csv')
cleaned_library = datasets.joinpath('library_cleaned.csv')
original_dataset = datasets.joinpath('vgsales.csv')

game_library_data = pd.read_csv(library)
game_library = pd.DataFrame(game_library_data)

data = pd.read_csv(original_dataset)
df = pd.DataFrame(data)
df = df.drop_duplicates(subset=['Name'], keep='first')
df.shape[0]

11493

## Authorization Step
___


In [6]:
# 'twitch_client_id' and 'twitch_client_secret' are imported from 'api_auth_keys.py'
twitch_oauth2_link = f"https://id.twitch.tv/oauth2/token?client_id={twitch_client_id}&client_secret={twitch_client_secret}&grant_type=client_credentials"
base_igdb_url = "https://api.igdb.com/v4"
post = requests.post(url=twitch_oauth2_link)
post_json_dumped = json.loads(post.text)
access_token = post_json_dumped['access_token']
# expires_in = post_json_dumped['expires_in']
# token_type = post_json_dumped['token_type']

### Game Titles / Chunking the data
---

In [12]:
game_titles = df['Name'].tolist()

cleaned_data = pd.read_csv(cleaned_library, encoding='utf-8')
clean_library = pd.DataFrame(cleaned_data)
clean_library[['id', 'name', 'genres', 'themes', 'summary']]

,id,name,genres,themes,summary
0,2181,Wii Sports,"simulation,sport",party,Bowl like a pro or go for that home run...\r\n...
1,358,Super Mario Bros.,"platformer,adventure",action,A side scrolling 2D platformer and first entry...
2,2183,Mario Kart Wii,racing,"action, comedy, party",Mario Kart Wii is a multiplayer-oriented racin...
3,2182,Wii Sports Resort,sport,party,Set off on a journey to a tropical island wher...
4,1076,New Super Mario Bros.,"platformer,adventure",action,New Super Mario Bros. is a side-scrolling 2.5D...
...,...,...,...,...,...
3676,2028,Elven Legacy,"rpg,strategy,tbs",fantasy,The shadows of destruction begin to loom on th...
3677,1593,Pillars of Eternity,"rpg,strategy,adventure,indie",fantasy,Prepare to be enchanted by a world where the c...
3678,1840,Neverwinter Nights 2: Mask of the Betrayer,"rpg,tactical",fantasy,Following the climactic battle against the Kin...
3679,78777,Legoland,"simulation,strategy","action, sandbox, educational",Legoland is an educational game that shows how...


In [16]:
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

In [17]:
# creates 115 chunks, each with 100 titles
chunks = list(chunk(game_titles, 100))
len(chunks)

115

### Creating a Data Pipeline
___

The first hurdle we must cross is setting up a connection between our project and the data source. We must create a way to reliably request data and validate that what's returned is appropriate to include in our game library.

IGDB's API has a rate limit of 4 requests/sec, so we must keep this in mind when establishing this connection as well.



In [18]:
FIELDS = ['id', 'release_dates', 'name', 'category', 'slug', 'platforms', 'genres', 'tags', 'age_ratings',
          'rating', 'rating_count', 'similar_games', 'themes', 'summary', 'involved_companies']

 #### Querying IGDB
---

IGDB uses a SQL-like language called APICalypse to construct queries for requesting data. Currently, there are 3 different ways of searching for titles in the database which we will refer to as:
    - `search`
    - `name equals`
    - `name like`

These names do not refer to the actual syntax of the search query, but instead how the search is conducted.

- `search` is the 'fuzziest' of the three, does not require titles to match exactly, but also returns less predictable results. `search` results are sorted by relevance descending, and hence, is incompatible with APICalypse's 'sort' keyword.
    Ex: `f id,name,release_dates.y,category,slug,platforms.name,genres.name,tags,age_ratings.rating,rating,rating_count,similar_games.name,themes.name,summary,involved_companies.company.name;where (name = "The Legend of Zelda: Tears of the Kingdom") & (category = (0,1,2,8,9,10,11)) & (version_parent = null);limit 25;offset 0;`

- `name equals` is best suited for titles that match exactly between our dataset and their respective entries in the database. Hence, its results are more limited, and more likely to return nothing.
    Ex: `f id,name,release_dates.y,category,slug,platforms.name,genres.name,tags,age_ratings.rating,rating,rating_count,similar_games.name,themes.name,summary,involved_companies.company.name;where (name = "The Legend of Zelda: Tears of the Kingdom") & (category = (0,1,2,8,9,10,11)) & (version_parent = null);limit 25;offset 0;`

- `name like` behaves a bit like both previous methods, and can be formatted to match on the beginning, end or entirety of a title.
    Ex: `f id,name,release_dates.y,category,slug,platforms.name,genres.name,tags,age_ratings.rating,rating,rating_count,similar_games.name,themes.name,summary,involved_companies.company.name;where (name ~*"The Legend of Zelda: Tears of the Kingdom"*) & (category = (0,1,2,8,9,10,11)) & (version_parent = null);limit 25;offset 0;`





In [125]:
# test sample of a chunk:
# for c in chunks:
#     print(c)
chunks[15]

('Dragon Ball Z: Taiketsu',
 "Disney's Lilo & Stitch",
 'Star Fox: Assault',
 'Naruto: Ultimate Ninja Storm',
 'NCAA Football 14',
 'Doko Demo Issyo',
 'Imagine: Fashion Designer New York',
 'Bully: Scholarship Edition',
 'Virtua Fighter',
 'Championship Manager 99/00',
 'Super Princess Peach',
 'NHL 99',
 'SoulCalibur III',
 'F1 2012',
 'World Tour Soccer',
 'Disney Universe',
 'Ridge Racer Revolution',
 'Hot Wheels Turbo Racing',
 'NCAA Football 99',
 'Mortal Kombat: Armageddon',
 'Aliens vs Predator',
 'Freekstyle',
 'Junior Brain Trainer',
 'Darksiders',
 'DiRT',
 'Simple 1500 Series Vol. 1: The Mahjong',
 'NBA Street V3',
 'Sleeping Dogs',
 'Mercenaries 2: World in Flames',
 'Worms Armageddon',
 'Hulk',
 'NFL Quarterback Club 98',
 'DJ Hero',
 'Dora the Explorer: Dora Saves the Mermaids',
 'Diablo II: Lord of Destruction',
 'Naruto Shippuden: Ultimate Ninja Storm 4',
 'Virtua Tennis 3',
 'Seek and Destroy',
 'Warhawk',
 'Apollo Justice: Ace Attorney',
 'Cosmic Ark',
 'SOCOM: U.S. 

In [14]:
passes, fails = [], []
# no release_dates
discards = []

In [19]:
# [[[[appended up to chunk 100]]]]
def request_chunks(start: int, stop: int, reset_passes: bool = True, display: bool = True, search_type: str = 'name equals'):
    """Automates post requests to IGDB for the slice of chunks specified"""
    # run collection process
    # reset passes with each run of this process, keep fails to retry later
    if reset_passes:
        passes = []
    for c in chunks[start:stop]:
        for i, j in enumerate(c):
            title = j
            search_type = search_type
            query = format_query(title, search_type=search_type)
            response = post_request(query, twitch_client_id, access_token)
            result = validate_response(response, title, exact_matches_only=True)
            # print(type(result))
            if isinstance(result, dict):
                # for some reason some of these don't have release_dates handling that here is easier
                if result.get('release_dates'):
                    if result.get('id') not in game_library.id.tolist():
                        if display:
                            print(f"PASS: {result.get('name')}")
                        parsed_response = parse_response(result)
                        passes.append(parsed_response)
                    else:
                        discards.append(result)
                elif not result.get('release_dates'):
                    discards.append(result)
            else:
                fails.append((title, search_type))
    if display:
        print(f"{len(passes)}, {len(fails)}, {len(discards)}")
    return passes, fails, discards

In [153]:
passes, fails, discards = request_chunks(101, len(chunks))

PASS: D1 Professional Drift Grand Prix Series
PASS: Darius II
PASS: RollerCoaster Tycoon 2: Triple Thrill Pack
PASS: The Sims 2: Bon Voyage
PASS: Divinity II: Flames of Vengeance
PASS: Renegade Racers
PASS: The King of Fighters: Maximum Impact - Maniax
PASS: Wand of Fortune 2: Jikuu ni Shizumu Mokushiroku
PASS: Combat Wings: The Great Battles of WWII
PASS: Tank Beat
PASS: Star Wars: Empire at War - Forces of Corruption
PASS: Tube Slider
PASS: Ghost Pirates of Vooju Island
PASS: Pharaoh
PASS: Pony Luv
PASS: Vantage Master Portable
PASS: Tropico 4: Modern Times
PASS: Kensei: Sacred Fist
PASS: Shanghai: True Valor
PASS: Tokyo Xanadu eX+
PASS: Vitamin Z Revolution
PASS: Natalie Brooks: Mystery at Hillcrest High
PASS: Treasure Chase
PASS: RayCrisis: Series Termination
PASS: Brandish: The Dark Revenant
PASS: Samurai Shodown V
PASS: Homeworld
PASS: Invizimals: The Lost Kingdom
PASS: Veggy World
PASS: Doki Majo Plus
PASS: Eureka Seven Vol. 2: The New Vision
PASS: Motto! SoniComi
PASS: Saint
PA

In [15]:
len(passes), len(fails), len(discards)

(0, 0, 0)

In [254]:
# remove passed titles that already exist in our library
# dupes = 0
# passes_temp = passes.copy()
# for i, j in enumerate(passes_temp):
#     game_id = j.get('id')
#     if game_id in game_library.id.to_list():
#         dupes += 1
#         passes.remove(j)
# len(passes), dupes

(7, 59)

In [155]:
[k.get('name') for k in passes]

['D1 Professional Drift Grand Prix Series',
 'Darius II',
 'RollerCoaster Tycoon 2: Triple Thrill Pack',
 'The Sims 2: Bon Voyage',
 'Divinity II: Flames of Vengeance',
 'Renegade Racers',
 'The King of Fighters: Maximum Impact - Maniax',
 'Wand of Fortune 2: Jikuu ni Shizumu Mokushiroku',
 'Combat Wings: The Great Battles of WWII',
 'Tank Beat',
 'Star Wars: Empire at War - Forces of Corruption',
 'Tube Slider',
 'Ghost Pirates of Vooju Island',
 'Pharaoh',
 'Pony Luv',
 'Vantage Master Portable',
 'Tropico 4: Modern Times',
 'Kensei: Sacred Fist',
 'Shanghai: True Valor',
 'Tokyo Xanadu eX+',
 'Vitamin Z Revolution',
 'Natalie Brooks: Mystery at Hillcrest High',
 'Treasure Chase',
 'RayCrisis: Series Termination',
 'Brandish: The Dark Revenant',
 'Samurai Shodown V',
 'Homeworld',
 'Invizimals: The Lost Kingdom',
 'Veggy World',
 'Doki Majo Plus',
 'Eureka Seven Vol. 2: The New Vision',
 'Motto! SoniComi',
 'Saint',
 'Steal Princess',
 'Legend: Hand of God',
 'Umineko no Naku Koro ni

In [162]:
[k.get('name') for k in discards]

['SingStar: Back to the 80s',
 "Crash 'N' Burn",
 "Final Fantasy Fables: Chocobo's Dungeon",
 'Disaster Report',
 'We Sing Robbie Williams',
 "Akiba's Trip: Undead & Undressed",
 'College Hoops 2K7',
 'Baja: Edge of Control',
 'Wild Wild Racing',
 "The Bard's Tale",
 'Conflict Zone',
 'de Blob 2',
 '50 Cent: Blood on the Sand',
 'Tenchu: Return From Darkness',
 'Neo Contra',
 'MVP Baseball',
 'Breakaway IV',
 "Now! That's What I Call Music: Dance & Sing",
 "Yu-Gi-Oh! 5D's World Championship 2011: Over the Nexus",
 'Senran Kagura 2: Deep Crimson',
 'Champion Jockey: G1 Jockey & Gallop Racer',
 'HBO Boxing',
 'Yoga Wii',
 "Cabela's Survival: Shadows of Katmai",
 'Prince of Persia: Revelations',
 'Front Mission 5: Scars of the War',
 'Wedding Dash',
 'Hot Shots Tennis: Get a Grip',
 'Summoner 2',
 'The Land Before Time: Big Water Adventure',
 'Shadow of Destiny',
 'ClayFighter 63 1/3',
 'Heroes of Ruin',
 'Bastard!! Utsuro Naru Kamigami no Utsuwa',
 'Angel Blade: Neo Tokyo Guardians',
 'T

In [256]:
# remove duplicates from the list of passes
# passes_copy = passes.copy()
# passed_ids = set([k.get('id') for k in passes])
# temp = []
# for i, j in enumerate(passes_copy):
#     if j.get('id') in passed_ids and j.get('id') not in temp:
#         temp.append(j.get('id'))
#     elif j.get('id') in passed_ids and j.get('id') in temp:
#         passes.remove(j)
# len(passes)

7

In [211]:
# clear temp lists
# passes_temp, passes_copy = [], []
# len(passes_temp), len(passes_copy)

(0, 0)

In [98]:
# [passes[i].get('name') for i, j in enumerate(fails)]
# fails[::-1]

[('The Adventures of Jimmy Neutron Boy Genius vs. Jimmy Negatron',
  'name equals'),
 ('Jikkyou Powerful Pro Yakyuu 2013', 'name equals'),
 ('Inazuma Eleven GO 3: Galaxy', 'name equals'),
 ('DS Bungaku Zenshuu', 'name equals'),
 ('Love Plus', 'name equals'),
 ('Tomba!', 'name equals'),
 ('Zoo Tycoon 2: Ultimate Collection', 'name equals'),
 ('NPPL: Championship Paintball 2009', 'name equals'),
 ("Jikkyou Powerful Pro Yakyuu '99 Ketteiban", 'name equals'),
 ('Littlest Pet Shop: Friends', 'name equals'),
 ("A.C.E.: Another Century's Episode R", 'name equals'),
 ('Create', 'name equals'),
 ('Reload: Target Down', 'name equals'),
 ('Sonic X: Game Boy Advance Video Volume 1', 'name equals'),
 ('And1 Streetball', 'name equals'),
 ('Fish Tycoon', 'name equals'),
 ('Imagine: Fashion Party', 'name equals'),
 ('Dynasty Warriors 7: Xtreme Legends', 'name equals'),
 ('Operation Armored Liberty', 'name equals'),
 ('Jikkyou Powerful Pro Yakyuu 2010', 'name equals'),
 ('My Virtual Tutor: Reading Adve

In [20]:
def append_rows(rows: list | dict):
    pre_exisiting_titles = game_library.id.tolist()
    newly_added_titles = new_data_df.id.tolist()

    with open(new_data, 'a', encoding='utf-8') as csv_file:
        dict_writer = csv.DictWriter(csv_file, FIELDS)
        if isinstance(rows, dict):
            game_id = rows.get('id')
            # check that name isn't already in library
            if game_id not in pre_exisiting_titles and game_id not in newly_added_titles:
                print(rows[i].get('name'))
                dict_writer.writerow(rows)
            else:
                pass
        elif isinstance(rows, list):
            print('///')
            for i in range(len(rows)):
                if isinstance(rows[i], dict):
                    game_id = rows[i].get('id')
                    if game_id not in pre_exisiting_titles and game_id not in newly_added_titles:
                        print(rows[i].get('name'))
                        dict_writer.writerow(rows[i])

In [156]:
# write new rows to csv
append_rows(passes)

///
D1 Professional Drift Grand Prix Series
Darius II
RollerCoaster Tycoon 2: Triple Thrill Pack
The Sims 2: Bon Voyage
Divinity II: Flames of Vengeance
Renegade Racers
The King of Fighters: Maximum Impact - Maniax
Wand of Fortune 2: Jikuu ni Shizumu Mokushiroku
Combat Wings: The Great Battles of WWII
Tank Beat
Star Wars: Empire at War - Forces of Corruption
Tube Slider
Ghost Pirates of Vooju Island
Pharaoh
Pony Luv
Vantage Master Portable
Tropico 4: Modern Times
Kensei: Sacred Fist
Shanghai: True Valor
Tokyo Xanadu eX+
Vitamin Z Revolution
Natalie Brooks: Mystery at Hillcrest High
Treasure Chase
RayCrisis: Series Termination
Brandish: The Dark Revenant
Samurai Shodown V
Homeworld
Invizimals: The Lost Kingdom
Veggy World
Doki Majo Plus
Eureka Seven Vol. 2: The New Vision
Motto! SoniComi
Saint
Steal Princess
Legend: Hand of God
Umineko no Naku Koro ni: Majo to Suiri no Rinbukyoku
Dance! It's Your Stage
Pebble Beach Golf Links
The Lord of the Rings Online: Mines of Moria
War Leaders: Cla

In [21]:
# preview new rows
new_data_ = pd.read_csv(new_data)
new_data_df = pd.DataFrame(new_data_)
new_data_df

,id,release_dates,name,category,slug,platforms,genres,tags,age_ratings,rating,rating_count,similar_games,themes,summary,involved_companies
0,121,2011.0,Minecraft,main_game,minecraft,"['Linux', 'PC (Microsoft Windows)', 'Mac']","['Simulator', 'Adventure']","['17', '21', '33', '35', '38', '268435469', '2...","['E10', 'Seven', 'CERO_A', 'USK_6', 'CLASS_IND...",84.545279,1527,"['The Elder Scrolls V: Skyrim', 'Dishonored', ...","['Fantasy', 'Survival', 'Sandbox', 'Kids', 'Op...",Minecraft focuses on allowing the player to ex...,['Mojang Studios']
1,135400,2018.0,Minecraft,port,minecraft--1,"['PC (Microsoft Windows)', 'Android', 'iOS', '...","['Simulator', 'Adventure']","['1', '17', '21', '33', '35', '38', '41', '268...","['Seven', 'E10', 'CERO_A', 'USK_6', 'GRAC_Twel...",79.003788,66,"['Life is Feudal: Your Own', 'Rising World', '...","['Action', 'Fantasy', 'Survival', 'Sandbox', '...",Minecraft focuses on allowing the player to ex...,['Mojang Studios']
2,119388,2023.0,The Legend of Zelda: Tears of the Kingdom,main_game,the-legend-of-zelda-tears-of-the-kingdom,['Nintendo Switch'],"['Role-playing (RPG)', 'Adventure']","['1', '17', '18', '38', '268435468', '26843548...","['CERO_B', 'GRAC_Twelve', 'CLASS_IND_Ten', 'E1...",96.989901,129,"['Cyberpunk 2077', 'Borderlands 3', 'Immortal:...","['Action', 'Fantasy', 'Science fiction', 'Open...",The Legend of Zelda: Tears of the Kingdom is t...,"['Nintendo EPD Production Group No. 3', 'Monol..."
3,2744,2006.0,Big Brain Academy,main_game,big-brain-academy,"['Nintendo DS', 'Wii U']",['Puzzle'],"['1', '34', '268435465', '536871186', '5368718...","['E', 'Three', 'CERO_A']",57.981056,8,"['Toby: The Secret Mine', 'Forgotton Anne', ""D...","['Action', 'Educational']",Big Brain Academy features 15 activities that ...,['Nintendo EAD Software Development Group No.4...
4,119295,2020.0,Microsoft Flight Simulator,main_game,microsoft-flight-simulator,"['PC (Microsoft Windows)', 'Xbox One', 'Xbox S...",['Simulator'],"['32', '268435469', '536870968', '536873753']","['E', 'Three', 'CERO_A']",93.519402,52,"['House Flipper', 'Cooking Simulator', ""Jet de...",['Non-fiction'],Microsoft Flight Simulator is the next generat...,"['Xbox Game Studios', 'Asobo Studio']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,5007,2005.0,Mini Desktop Racing,main_game,mini-desktop-racing,"['Wii', 'PC (Microsoft Windows)', 'PlayStation...",['Racing'],['268435466'],['E'],NaN,0,"['Rise: Race the Future', 'LA Street Racing', ...",NaN,Mini Desktop Racing lets you loose with a craz...,"['Data Design Interactive', 'Metro 3D', 'Consp..."
768,221225,2008.0,Neo Angelique Special,expanded_game,neo-angelique-special,['PlayStation Portable'],['Visual Novel'],"['17', '31', '44', '268435490', '536871874', '...",['CERO_B'],NaN,0,"[""Your Memories Off: Girl's Style"", 'Angelique...","['Fantasy', 'Drama', 'Romance']",Fully voiced and added omake events for the ch...,['Koei']
769,23671,2012.0,Outdoors Unleashed: Africa 3D,main_game,outdoors-unleashed-africa-3d,['Nintendo 3DS'],"['Shooter', 'Simulator', 'Adventure']","['1', '268435461', '268435469', '268435487', '...","['Sixteen', 'T', 'CERO_B']",NaN,0,"['Star Control: Origins', 'Survivalist', 'Chil...",['Action'],A downloadable game for 3DS in which you shoot...,['Teyon']
770,49359,2002.0,Planet Monsters,main_game,planet-monsters,['Game Boy Advance'],['Puzzle'],"['1', '268435465']",['E'],NaN,0,"['Digging Jim', 'Toby: The Secret Mine', ""Don'...",['Action'],There is only one rule with this game - Be the...,NaN


Based on our filtering, this round of querying only returned 800 out of roughly 11,500 titles. This is because we were only allowing titles that were an exact match (search_type = 'name equals') to those in our original dataset. Based on prior analysis and testing, we have reason to believe many of the titles that did not pass do match those in our original dataset, but just may have slightly different formatting in their titles.
There is also the possibility that a subset of the titles that did not pass did so because their entries in IGDB lack any data for the fields we need for training. There's not much that can be done about these, unless we consult other data sources, but that would host its own set of difficulties that may not be worth our time.

The next step is to conduct another round of queries on the failed titles from this round, but with the next search_type, 'search'.
The behavior of 'search' is not entirely clear, and may cause issues in our pipeline, so we must be careful and validate the results after each batch of chunks.

In [171]:
# get list of failed titles to retry
failed_titles = [i[0] for i in fails]
# failed_titles
# chunk the new list
failed_chunks = list(chunk(failed_titles, 100))
#
failed_chunks[0]

('NBA',
 "Pro Yakyuu Greatest Nine '97",
 'Dorabase DS: Dramatic Stadium',
 'EVE: burst error',
 "Backyard NFL Football '09",
 'Big League Sports',
 "Magician's Quest: Town of Magic",
 'Momotarou Dentetsu 16',
 'Robotech: The Macross Saga',
 'Yu Yu Hakusho: Dark Tournament',
 'Gekijouban Macross F: Itsuwarino Utahime - Hybrid Pack',
 'World Poker Tour',
 'GRID Autosport',
 'Ghostbusters II',
 'RealSports Baseball',
 'Quake III: Revolution',
 'Pac-Man World 3',
 'Naruto: Clash of Ninja (JP sales)',
 'NOëL: NOT DiGITAL',
 'Transformer: Rise of the Dark Spark',
 'Dragon Ball Z Hyper Dimension',
 'Maximo vs Army of Zin',
 "The Fairly OddParents: Breakin' Da Rules",
 'Namco Museum Megamix',
 'Valkyria Chronicles III: Unrecorded Chronicles',
 "Cruis'n Exotica",
 "The King of Fighters '96",
 'WRC: FIA World Rally Championship',
 'Famicom Mini: Ganbare Goemon! Karakuri Douchuu',
 'J-League Winning Eleven 10 + Europa League 06-07',
 'Tales of Phantasia: Narikiri Dungeon X',
 'JSRF: Jet Set Radi

In [196]:
for c in failed_chunks[0:1]:
    for i, j in enumerate(c):
        title = j
        search_type = 'search'
        query = format_query(title, search_type=search_type)
        response = post_request(query, twitch_client_id, access_token)
        result = validate_response(response, title, exact_matches_only=False)

        if isinstance(result, dict):
            print(result.json())
            print('\n\n')
            # # for some reason some of these don't have release_dates handling that here is easier
            # if result.get('release_dates'):
            #     if result.get('id') not in game_library.id.tolist():
            #         if display:
            #             print(f"PASS: {result.get('name')}")
            #         parsed_response = parse_response(result)
            #         passes.append(parsed_response)
            #     else:
            #         discards.append(result)
            # elif not result.get('release_dates'):
            #     discards.append(result)

        elif isinstance(result, list):
            for r in result:
                print(r.json())
                print('\n\n')
            # for r in result:
            #     resp = r.json()
            #     print(resp)
            #     if resp.get('release_dates'):
            #         if resp.get('id') not in game_library.id.tolist():
            #             if display:
            #                 print(f"PASS: {resp.get('name')}")
            #             parsed_response = parsed_response(resp)
            #             passes.append(resp)
            #         else:
            #             discards.append(resp)
            #     elif not r.get('release_dates'):
            #         discards.append(resp)
        else:
            fails.append((title, search_type))
if display:
    print(f"{len(passes)}, {len(fails)}, {len(discards)}")

[{'id': 45573, 'category': 0, 'genres': [{'id': 14, 'name': 'Sport'}], 'involved_companies': [{'id': 102777, 'company': {'id': 1, 'name': 'Electronic Arts'}}], 'name': 'NBA Showdown', 'platforms': [{'id': 19, 'name': 'Super Nintendo Entertainment System'}, {'id': 29, 'name': 'Sega Mega Drive/Genesis'}], 'release_dates': [{'id': 201522, 'y': 1994}, {'id': 201523, 'y': 1993}, {'id': 201524, 'y': 1994}], 'similar_games': [{'id': 43718, 'name': 'NBA In the Zone 2000'}, {'id': 46067, 'name': 'Bulls vs Lakers and the NBA Playoffs'}, {'id': 46284, 'name': 'Bulls Versus Blazers and the NBA Playoffs'}, {'id': 72053, 'name': "NBA in the Zone '99"}, {'id': 96209, 'name': 'FIFA 19'}, {'id': 103218, 'name': 'NBA 2K19'}, {'id': 103275, 'name': 'NBA Live 19'}, {'id': 114285, 'name': 'NBA 2K20'}, {'id': 114286, 'name': 'Madden NFL 20'}, {'id': 114287, 'name': 'FIFA 20'}], 'slug': 'nba-showdown', 'summary': "The final entry in Electronic Arts' basketball series based on the old NBA playoffs engine befo

KeyboardInterrupt: 

In [179]:
print(failed_chunks[0])

('NBA', "Pro Yakyuu Greatest Nine '97", 'Dorabase DS: Dramatic Stadium', 'EVE: burst error', "Backyard NFL Football '09", 'Big League Sports', "Magician's Quest: Town of Magic", 'Momotarou Dentetsu 16', 'Robotech: The Macross Saga', 'Yu Yu Hakusho: Dark Tournament', 'Gekijouban Macross F: Itsuwarino Utahime - Hybrid Pack', 'World Poker Tour', 'GRID Autosport', 'Ghostbusters II', 'RealSports Baseball', 'Quake III: Revolution', 'Pac-Man World 3', 'Naruto: Clash of Ninja (JP sales)', 'NOëL: NOT DiGITAL', 'Transformer: Rise of the Dark Spark', 'Dragon Ball Z Hyper Dimension', 'Maximo vs Army of Zin', "The Fairly OddParents: Breakin' Da Rules", 'Namco Museum Megamix', 'Valkyria Chronicles III: Unrecorded Chronicles', "Cruis'n Exotica", "The King of Fighters '96", 'WRC: FIA World Rally Championship', 'Famicom Mini: Ganbare Goemon! Karakuri Douchuu', 'J-League Winning Eleven 10 + Europa League 06-07', 'Tales of Phantasia: Narikiri Dungeon X', 'JSRF: Jet Set Radio Future', 'The Last Airbender'

We now have 27 chunks of titles to retry querying with search_type = 'search'

In [51]:
game_library

,id,release_dates,name,category,slug,platforms,genres,tags,age_ratings,rating,rating_count,similar_games,themes,summary,involved_companies
0,2181,2006.0,Wii Sports,main_game,wii-sports,['Wii'],"['Simulator', 'Sport']","['40', '268435469', '268435470', '536870932', ...","['E', 'Seven', 'CLASS_IND_L', 'CERO_B', 'USK_1...",74.803922,301,"['Tekken 3', 'Tekken 5', 'Journey', 'Wii Sport...",['Party'],Bowl like a pro or go for that home run...\r\n...,"['Nintendo', 'Nintendo EAD Software Developmen..."
1,358,1987.0,Super Mario Bros.,main_game,super-mario-bros,"['Wii', 'Nintendo Entertainment System', 'Nint...","['Platform', 'Adventure']","['1', '268435464', '268435487', '536870928', '...","['E', 'Three', 'CLASS_IND_L', 'CERO_A', 'USK_0...",83.948468,1041,"['Zelda II: The Adventure of Link', 'The Legen...",['Action'],A side scrolling 2D platformer and first entry...,"['Nintendo', 'Nintendo R&D4', 'Playtronic']"
2,2183,2008.0,Mario Kart Wii,main_game,mario-kart-wii,['Wii'],['Racing'],"['1', '27', '40', '268435466', '536870937', '5...","['E', 'Three', 'CLASS_IND_L', 'CERO_A', 'ACB_G...",80.180221,458,"['Wreckfest', 'Cocoto Kart Racer', 'Borderland...","['Action', 'Comedy', 'Party']",Mario Kart Wii is a multiplayer-oriented racin...,"['Nintendo', 'Nintendo EAD']"
3,2182,2009.0,Wii Sports Resort,main_game,wii-sports-resort,['Wii'],['Sport'],"['40', '268435470', '536870942', '536870970', ...","['E', 'Seven', 'CLASS_IND_L', 'CERO_B', 'USK_1...",78.657933,137,"['Wii Sports', 'Wii Play', 'Wii Fit', 'Wii Fit...",['Party'],Set off on a journey to a tropical island wher...,"['Nintendo', 'Nintendo EAD Software Developmen..."
4,1076,2006.0,New Super Mario Bros.,main_game,new-super-mario-bros,"['Nintendo DS', 'Wii U']","['Platform', 'Adventure']","['1', '268435464', '268435487', '536870928', '...","['E', 'CLASS_IND_L', 'CERO_A', 'USK_0', 'ACB_P...",82.089739,287,"['Super Mario Bros.', 'The Legend of Zelda: Tw...",['Action'],New Super Mario Bros. is a side-scrolling 2.5D...,"['Nintendo EAD', 'Nintendo', 'Nintendo', 'iQue']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12916,63706,2001.0,Densha de GO! 3 Tsuukinhen,main_game,densha-de-go-3-tsuukinhen,"['PlayStation 2', 'Arcade']",['Simulator'],"['32', '268435469']",NaN,NaN,0,"['Industry Giant', 'Sovereignty: Crown of King...",['Non-fiction'],Densha de Go! 3 Tsuukin-hen (loosely translate...,"['Taito', 'Taito GM R&D Sect. #2']"
12917,5642,2009.0,Guilty Gear XX Accent Core Plus,expanded_game,guilty-gear-xx-accent-core-plus,"['Wii', 'PlayStation 2', 'PlayStation 3', 'Xbo...",['Fighting'],"['17', '18', '268435460', '536870990', '536871...","['GRAC_Twelve', 'M']",79.738110,18,"['The Legend of Heroes: Trails of Cold Steel',...","['Fantasy', 'Science fiction']",Λ Core Plus also features a new Story mode. Th...,"['Arc System Works', 'Aksys Games']"
12918,38477,2008.0,Higurashi Daybreak Portable,port,higurashi-daybreak-portable,['PlayStation Portable'],['Fighting'],"['1', '268435460']",['CERO_B'],NaN,0,"['Dragon Ball Z: Budokai Tenkaichi 3', 'One Pi...",['Action'],This game is an enhanced port of the doujin ga...,"['Cavia', 'Alchemist']"
12919,49457,2006.0,WinX Club: Quest for the Codex,main_game,winx-club-quest-for-the-codex,['Game Boy Advance'],['Adventure'],"['1', '34', '268435487', '536875054', '5368752...",['E'],NaN,0,"['Cyberpunk 2077', ""Don't Knock Twice"", ""Marve...","['Action', 'Educational']",Welcome to the magical world of Winx Club wher...,NaN


In [36]:
def search_library(col_name: str, search_term: str):
    notna_library = game_library[game_library['name'].notna()]
    search_result = notna_library[notna_library[col_name].str.contains(search_term)]
    return pd.DataFrame(search_result)

In [163]:
search_library('name', 'Nancy Drew')

,id,release_dates,name,category,slug,platforms,genres,tags,age_ratings,rating,rating_count,similar_games,themes,summary,involved_companies
4579,47907,2008.0,Nancy Drew: The Mystery of the Clue Bender Soc...,main_game,nancy-drew-the-mystery-of-the-clue-bender-society,['Nintendo DS'],['Adventure'],"['1', '43', '268435487', '536871127', '5368718...",['E10'],NaN,0,"['Savage Lands', 'GreedFall', 'Blacksad: Under...","['Action', 'Mystery']","The Clue Bender Society, a group consisting of...","['Majesco Entertainment', 'Gorilla Systems Cor..."
5665,47916,2008.0,Nancy Drew: The Hidden Staircase,main_game,nancy-drew-the-hidden-staircase,['Nintendo DS'],"['Puzzle', 'Adventure']","['268435465', '268435487', '536871874', '53687...",['E'],NaN,0,"['Rusty Lake: Roots', ""Don't Knock Twice"", 'Qu...",NaN,NANCY'S ON THE CASE! Family heirlooms are disa...,['Majesco Entertainment']
6106,5050,2008.0,Nancy Drew: The White Wolf of Icicle Creek,main_game,nancy-drew-the-white-wolf-of-icicle-creek,"['Wii', 'PC (Microsoft Windows)']","['Point-and-click', 'Adventure']","['268435458', '268435487', '536871127', '53687...",['E'],70.007982,10,"['Nancy Drew: Legend of the Crystal Skull', 'M...",NaN,Follow a Trail to Hidden Secrets and Sabotage!,"['Her Interactive', 'Sega']"
